In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType
import torch
import numpy as np

In [2]:
MODEL_NAME = "Salesforce/codet5-base"
OUTPUT_DIR = "outputs/codet5-lora"
MAX_INPUT = 128
MAX_OUTPUT = 64
EPOCHS = 1     #inc later
BATCH = 4
LR = 5e-4

In [3]:
dataset = load_dataset("code_x_glue_ct_code_to_text", "python")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

python/train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

python/train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

python/validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

python/test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})


In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    inputs = [f"summarize: {code}" for code in batch["code"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT, truncation=True, padding="max_length")
    labels = tokenizer(batch["docstring"], max_length=MAX_OUTPUT, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess, batched=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/251820 [00:00<?, ? examples/s]

Map:   0%|          | 0/13914 [00:00<?, ? examples/s]

Map:   0%|          | 0/14918 [00:00<?, ? examples/s]

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [6]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=16,
    lora_alpha=32,     # scaling
    lora_dropout=0.1,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 224,651,520 || trainable%: 0.7877


In [7]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=torch.cuda.is_available(),
)

In [8]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"].select(range(40000)),
    eval_dataset=tokenized["validation"].select(range(1000)),
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer.train()

/tmp/ipython-input-43671964.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mehultotala (mehultotala-veermata-jeejabai-technological-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.013700,0.065405


TrainOutput(global_step=10000, training_loss=0.04172508212029934, metrics={'train_runtime': 2212.6124, 'train_samples_per_second': 18.078, 'train_steps_per_second': 4.52, 'total_flos': 6143937085440000.0, 'train_loss': 0.04172508212029934, 'epoch': 1.0})

In [9]:
tokenizer.save_pretrained(OUTPUT_DIR)

print("Training done. Model + LoRA adapter saved in:", OUTPUT_DIR)

Training done. Model + LoRA adapter saved in: outputs/codet5-lora


In [10]:
def summarize(code):
    inputs = tokenizer(f"summarize: {code}", return_tensors="pt", truncation=True).to(model.device)
    output = model.generate(**inputs, max_length=MAX_OUTPUT)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print("\n Test prediction:")
print(summarize("def factorial(n): return 1 if n==0 else n*factorial(n-1)"))


 Test prediction:
factorial(n) returns 1 if n==0 else n*factorial(n-1) else n*factorial(n-1) else factorial(n-1) return 1 if n==0 else n*factorial(n-1) else n*factorial(n
